# Normalizing Flow

标准化流模型（normalizing flow）是一类特殊的神经网络模型（见阅读材料），其最大的特点是具有可逆性。对于一个 $d$ 维的输入 $x\in\mathbb{R}^d$，流模型可以看作是一个 $\mathbb{R}^d\rightarrow\mathbb{R}^d$ 的映射 $f_\theta$，且 $f_\theta^{-1}$ 存在，其中 $\theta$ 是该映射的参数。我们一般希望 $z=f_\theta(x)$ 和 $x=f_\theta^{-1}(z)$ 都可高效地进行计算。

流模型中一类常见的实现被称为 Real NVP，它按如下的方式定义一个基础的可逆映射 $f_\theta$：对于 $d$ 维输入 $x=(x_1,\ldots,x_d)'$，首先固定一个整数 $0<k<d$，然后令 $z=(z_1,\ldots,z_d)'=f_\theta(x)$，其中

$$\begin{align*}
z_{1:k} & =x_{1:k},\\
z_{(k+1):d} & =x_{(k+1):d}\odot\exp(\sigma(x_{1:k}))+\mu(x_{1:k}).
\end{align*}$$

换言之，$x$ 的前 $k$ 个元素保持不变，而剩下的元素 $x_{(k+1):d}$ 将乘以一个等长的向量 $v$，再加上一个等长的向量 $u$ 得到 $z_{(k+1):d}$，其中 $u$ 和 $v$ 又是前 $k$ 个元素 $x_{1:k}$ 的函数，$v=\exp(\sigma(x_{1:k}))$，$u=\mu(x_{1:k})$，此处 $\mu(\cdot)$ 和 $\sigma(\cdot)$ 是两个 $\mathbb{R}^k\rightarrow\mathbb{R}^{d-k}$ 的前馈神经网络。 

**在实际模型实现中，输入数据是一个 $n\times d$ 的矩阵 $X$，而 $f_\theta$ 将对 $X$ 的每一行进行上述变换。**

Real NVP 还有一个重要的性质，就是其雅各比矩阵的行列式具有简单的形式：

$$\log\left[\left|\det\left(\frac{\partial f_\theta}{\partial x}\right)\right|\right]=\sum_{i=1}^{d-k}\sigma_i(x_{1:k}),$$

其中 $\sigma_i(x_{1:k})$ 是 $\sigma(x_{1:k})$ 输出的第 $i$ 个元素。

**(a) 在以下小问中我们始终让 $k=\lfloor\frac{d}{2}\rfloor$，其中 $\lfloor x\rfloor$ 表示不超过 $x$ 的最大整数。请编写一个简单的函数用来计算 $k$。（3分）**

In [108]:
def get_k(d):
    k=math.floor(d/2)
    return k
    # 在此处完成函数实现

**(b) $\mu(\cdot)$ 和 $\sigma(\cdot)$ 具有相同的结构，因此我们可以定义一个统一的类，然后从这个类中生成两个前馈神经网络对象。要求该前馈神经网络的类具有两个隐藏层（不包括输入层和输出层），其神经元数量分别为32和16，使用 ReLU 作为激活函数。完成以下模块的构建。（7分）**

In [97]:
import torch.nn as nn

class FeedForward(nn.Module):
    def __init__(self, d, k):
        super(FeedForward, self).__init__()
        self.fc1 = torch.nn.Linear(in_features = k , out_features = 32)
        self.fc2 = torch.nn.Linear(in_features = 32, out_features = 16)
        self.fc3 = torch.nn.Linear(in_features = 16, out_features = d-k)
    
    def forward(self, x):
        
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        
        return x

# 测试结果
torch.manual_seed(2023)
mu_fn = FeedForward(d=5, k=2)
sigma_fn = FeedForward(d=5, k=2)
x1k = torch.randn(3, 2)

print(mu_fn)
print(sigma_fn)
print(mu_fn(x1k))
print(sigma_fn(x1k))

FeedForward(
  (fc1): Linear(in_features=2, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=3, bias=True)
)
FeedForward(
  (fc1): Linear(in_features=2, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=3, bias=True)
)
tensor([[0.0772, 0.0000, 0.1758],
        [0.1746, 0.0000, 0.1949],
        [0.1221, 0.0000, 0.1793]], grad_fn=<ReluBackward0>)
tensor([[0.1531, 0.0000, 0.3252],
        [0.1426, 0.0000, 0.2933],
        [0.1256, 0.0000, 0.3138]], grad_fn=<ReluBackward0>)


**(c) 利用以上模块，编写一个 Real NVP 的模块类，用来实现标准化流模型。该模块类的 `forward()` 函数接收一个 $n\times d$ 的参数 `x`，然后返回变换的结果 `z`（$n\times d$ 的矩阵）以及雅各比矩阵的对数行列式取值 `logdet`（$n\times 1$ 的向量）。（15分）**

In [122]:
class RealNVP(nn.Module):
    def __init__(self, input_dim):
        super(RealNVP, self).__init__()
        self.mu = FeedForward(d=input_dim, k=get_k(input_dim))
        self.sig = FeedForward(d=input_dim, k = get_k(input_dim))

    def forward(self, x):
        
        d = x.shape[1]
        k = get_k(d)
        
        x1 = x[:,:k]
        x2 = x[:,k:]
        
        # get z1
        z1 = x1
        
        # get z2
        sig_out = self.sig(x1)
        X2Sig = sig_out * x2
        
        mu_out = self.mu(x1)
        X2SigMu = X2Sig + mu_out
        
        z2 = X2SigMu
        
        # concat [Z1:Z2]
        z = torch.cat((z1,z2),1)
        
        # logdet
        logdet = torch.sum(sig_out,dim=1)
        
        return z,logdet

# 测试结果
torch.manual_seed(2023)
n = 5
d = 7
x = torch.randn(n, d)
model = RealNVP(input_dim=d)
z, logdet = model(x)

print(x)
print(z)
print(logdet)

tensor([[ 0.4305, -0.3499,  0.4749,  0.9041, -0.7021,  1.5963,  0.4228],
        [-0.6940,  0.9672,  1.5569, -2.3860,  0.6994, -1.0325, -2.6043],
        [ 0.9337, -0.1050,  0.7427, -1.3397, -0.3649, -0.2325,  0.3958],
        [ 0.8536, -0.4204, -1.4516,  1.0055, -0.1263, -0.3242, -1.2767],
        [ 0.2009,  0.0190,  0.3041, -0.9213,  0.9191, -2.4946, -0.2740]])
tensor([[ 0.4305, -0.3499,  0.4749,  0.3313,  0.0542,  0.3460,  0.0000],
        [-0.6940,  0.9672,  1.5569,  0.1725,  0.0000, -0.0093,  0.0000],
        [ 0.9337, -0.1050,  0.7427,  0.4098,  0.0075,  0.1292,  0.0000],
        [ 0.8536, -0.4204, -1.4516,  0.1708,  0.2882, -0.0328,  0.0000],
        [ 0.2009,  0.0190,  0.3041,  0.2833,  0.0825, -0.1165,  0.0000]],
       grad_fn=<CatBackward0>)
tensor([0.1158, 0.1513, 0.1030, 0.2318, 0.1394], grad_fn=<SumBackward1>)


**(d) 根据阅读材料中的介绍，我们用一个流模型 $f_\theta$ 来表达一个 $d$ 维的密度函数，其中 $p_Z$ 是 $d$ 维标准正态分布的密度函数：**

$$
\log[p_{X}(x)]=\log\left[\left|\det\left(\frac{\partial f_\theta}{\partial x}\right)\right|\right]+\log[p_{Z}(f_\theta(x))].
$$

**请利用上面建立的 Real NVP 模型，计算该分布模型在 (c) 中数据 `x` 上的对数似然函数值。（5分）**

In [128]:
from torch.distributions import MultivariateNormal

def log_standard_normal_pdf(x):
    """
    计算 d 维标准正态分布的概率密度函数
    :param x: d 维向量或包含多个 d 维向量的张量
    :return: 对应的概率密度
    """
    d = x.size(-1)
    # 创建一个多维标准正态分布（均值为0，协方差矩阵为单位矩阵）
    distribution = MultivariateNormal(torch.zeros(d), torch.eye(d))
    # 计算概率密度
    return distribution.log_prob(x)

x.shape
logPz = log_standard_normal_pdf(x)
logPx = logdet + logPz
print(logPx)


tensor([ -8.6021, -15.2171,  -8.1161,  -9.0880, -10.3555],
       grad_fn=<AddBackward0>)


**(e) 在上面定义的 Real NVP 模型中，$f_\theta$ 的变换输出 $z$ 的前 $k$ 个分量与输入 $x$ 的前 $k$ 个分量是完全一致的，这会使得模型的表达能力受到限制。结合阅读材料，请用文字简述有哪些方法可以增强模型的表达能力，使其能刻画更复杂的非线性关系。（5分）**

比如可以利用coupling flows，先选取一块保持不变，再利用神经网络训练另外一块在合并